
# Algorithmic Trading Machine Learning Project

This is a machine learning project that will take in S&P 500 stock price data and use an unsupervised learning trading strategy. This is strictly for educational purposes and is to help me learn machine learning.

1. Install packages.

In [33]:
# Packages installed:
# pandas
# pandas_ta
# numpy
# matplotlib
# statsmodels
# pandas_datareader
# datetime
# yfinance
# sklearn
# PyPortfolioOpt

!pip install pandas pandas_ta numpy matplotlib statsmodels pandas_datareader datetime yfinance sklearn PyPortfolioOpt -q


You should consider upgrading via the 'c:\users\kyle\onedrive\desktop\project folders\algoirthmic trading\my_virtual_env\scripts\python.exe -m pip install --upgrade pip' command.


2. Download S&P 500 data

In [51]:
from statsmodels.regression.rolling import RollingOLS
import pandas_datareader.data as web
import matplotlib.pyplot as plt
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
import pandas_ta
import warnings
warnings.filterwarnings('ignore')

# Read the S&P 500 list of companies
sp500 = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]

# Remove the dot from the symbol names
sp500['Symbol'] = sp500['Symbol'].str.replace('.', '-')

# Get the list of symbols
symbols_list = sp500['Symbol'].unique().tolist()
symbols_list

# Define the start and end dates
end_date = '2023-09-27'
start_date = pd.to_datetime(end_date) - pd.DateOffset(365 * 8)

# Download the data
df = yf.download(tickers = symbols_list, 
                 start=start_date, 
                 end=end_date).stack()

df.index.names = ['date', 'ticker']
df.columns = df.columns.str.lower()


[*********************100%%**********************]  503 of 503 completed

3 Failed downloads:
['SOLV', 'GEV', 'VLTO']: YFChartError("%ticker%: Data doesn't exist for startDate = 1443499200, endDate = 1695787200")


2. Technical indicators for all the stocks

In [52]:
# Garman-Klass volatility
df['garman_klass_vol'] = ((np.log(df['high']) - np.log(df['low'])) **2/2 - ((2 * np.log(2) - 1) * (np.log(df['adj close']) - np.log(df['open'])) ** 2))

# RSI
df['rsi'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.rsi(close = x, length = 20))

# Bolinger Bands
df['bb_low'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 0])
df['bb_med'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 1])
df['bb_high'] = df.groupby(level = 1)['adj close'].transform(lambda x: pandas_ta.bbands(close = np.log1p(x), length = 20).iloc[:, 2])

# ATR
def compute_atr(stock_data):
    atr = pandas_ta.atr(high = stock_data['high'],
                        low = stock_data['low'],
                        close = stock_data['adj close'],
                        length = 14)
    return atr.sub(atr.mean()).div(atr.std())
df['atr'] = df.groupby(level = 1, group_keys = False).apply(compute_atr)

# MACD
def compute_macd(close):
    macd = pandas_ta.macd(close = close, length = 20).iloc[:,0]
    return macd.sub(macd.mean()).div(macd.std())

df['macd'] = df.groupby(level = 1, group_keys = False)['adj close'].apply(compute_macd)

# Volume
df['dollar_vol'] = (df['adj close'] * df['volume']) / 1e6
df

Price               adj close       close        high         low        open  \
date       ticker                                                               
2015-09-29 A        31.425230   33.740002   34.060001   33.240002   33.360001   
           AAL      37.361618   39.180000   39.770000   38.790001   39.049999   
           AAPL     24.651136   27.264999   28.377501   26.965000   28.207500   
           ABBV     36.334896   52.790001   54.189999   51.880001   53.099998   
           ABT      33.478710   39.500000   40.150002   39.029999   39.259998   
...                       ...         ...         ...         ...         ...   
2023-09-26 XYL      88.736305   89.519997   90.849998   89.500000   90.379997   
           YUM     122.211006  124.010002  124.739998  123.449997  124.239998   
           ZBH     111.534821  112.459999  117.110001  112.419998  116.769997   
           ZBRA    223.960007  223.960007  226.649994  222.580002  225.970001   
           ZTS     175.547714  176.869995  178.449997  176.270004  176.580002   

Price                   volume  garman_klass_vol        rsi    bb_low  \
date       ticker                                                       
2015-09-29 A         2252400.0         -0.001082        NaN       NaN   
           AAL       7478800.0         -0.000443        NaN       NaN   
           AAPL    293461600.0         -0.005712        NaN       NaN   
           ABBV     12842800.0         -0.054656        NaN       NaN   
           ABT      12287500.0         -0.009402        NaN       NaN   
...                        ...               ...        ...       ...   
2023-09-26 XYL       1322400.0         -0.000018  26.146735  4.485761   
           YUM       1500600.0         -0.000051  36.057175  4.811707   
           ZBH       3610500.0          0.000022  31.893224  4.745884   
           ZBRA       355400.0          0.000133  29.494977  5.400991   
           ZTS       1463200.0          0.000062  42.623488  5.156108   

Price                bb_med   bb_high       atr      macd   dollar_vol  
date       ticker                                                       
2015-09-29 A            NaN       NaN       NaN       NaN    70.782188  
           AAL          NaN       NaN       NaN       NaN   279.420069  
           AAPL         NaN       NaN       NaN       NaN  7234.161929  
           ABBV         NaN       NaN       NaN       NaN   466.641803  
           ABT          NaN       NaN       NaN       NaN   411.369651  
...                     ...       ...       ...       ...          ...  
2023-09-26 XYL     4.567684  4.649607 -2.967042 -2.159188   117.344890  
           YUM     4.841672  4.871637 -2.813230 -1.363696   183.389836  
           ZBH     4.785551  4.825217 -2.109952 -0.881067   402.696470  
           ZBRA    5.539167  5.677342 -0.057389 -1.600791    79.595386  
           ZTS     5.214922  5.273735 -0.720920 -1.188278   256.861415  

[987967 rows x 14 columns]

3. Filter top 150 most liquid

In [98]:
# Define the list of columns to keep
last_cols = [c for c in df.columns.unique(0) if c not in ['dollar_vol', 'volume', 'open', 'high', 'low', 'close']]

data = pd.concat([df.unstack('ticker')['dollar_vol'].resample('M').mean().stack('ticker').to_frame('dollar_vol'),
df.unstack()[last_cols].resample('M').last().stack('ticker')], axis = 1).dropna()

data

dollar_vol   adj close       atr   bb_high    bb_low  \
date       ticker                                                          
2015-11-30 A        135.740909   38.950897 -0.948797  3.689080  3.544191   
           AAL      287.915785   39.429939  1.473839  3.827635  3.672028   
           AAPL    4023.984070   26.854139 -0.326059  3.368302  3.281679   
           ABBV     337.563903   40.393524  0.572009  3.823191  3.726695   
           ABT      211.659048   38.293568  0.553865  3.699763  3.656056   
...                        ...         ...       ...       ...       ...   
2023-09-30 OTIS     154.361752   78.356506 -2.269283  4.460712  4.370137   
           ABNB    1633.500725  132.279999 -1.006939  5.024801  4.857047   
           CEG      196.670369  107.862030 -0.705546  4.732493  4.652147   
           GEHC     212.275850   66.130219 -0.905989  4.270508  4.155436   
           KVUE     670.804320   20.006195 -2.650790  3.164168  3.014275   

                     bb_med  garman_klass_vol      macd        rsi  
date       ticker                                                   
2015-11-30 A       3.616636         -0.002098  0.567157  73.421554  
           AAL     3.749832         -0.000966 -0.418772  40.719000  
           AAPL    3.324991         -0.003307 -0.142789  55.537348  
           ABBV    3.774943         -0.059549  0.145677  49.376849  
           ABT     3.677909         -0.011216  0.335557  56.962557  
...                     ...               ...       ...        ...  
2023-09-30 OTIS    4.415425         -0.000097 -1.534536  33.116256  
           ABNB    4.940924          0.000213 -0.037854  44.494127  
           CEG     4.692320          0.000131  0.366876  55.245482  
           GEHC    4.212972          0.000185 -1.116463  40.922337  
           KVUE    3.089221         -0.000159 -1.435621  35.706312  

[46648 rows x 9 columns]

In [100]:
data['dollar_vol'].unstack('ticker').rolling(5*12).mean().stack()

dollar_vol   adj close       atr   bb_high    bb_low  \
date       ticker                                                         
2015-11-30 A              NaN   38.950897 -0.948797  3.689080  3.544191   
           AAL            NaN   39.429939  1.473839  3.827635  3.672028   
           AAPL           NaN   26.854139 -0.326059  3.368302  3.281679   
           ABBV           NaN   40.393524  0.572009  3.823191  3.726695   
           ABT            NaN   38.293568  0.553865  3.699763  3.656056   
...                       ...         ...       ...       ...       ...   
2023-09-30 OTIS           NaN   78.356506 -2.269283  4.460712  4.370137   
           ABNB           NaN  132.279999 -1.006939  5.024801  4.857047   
           CEG            NaN  107.862030 -0.705546  4.732493  4.652147   
           GEHC           NaN   66.130219 -0.905989  4.270508  4.155436   
           KVUE           NaN   20.006195 -2.650790  3.164168  3.014275   

                     bb_med  garman_klass_vol      macd        rsi  \
date       ticker                                                    
2015-11-30 A       3.616636         -0.002098  0.567157  73.421554   
           AAL     3.749832         -0.000966 -0.418772  40.719000   
           AAPL    3.324991         -0.003307 -0.142789  55.537348   
           ABBV    3.774943         -0.059549  0.145677  49.376849   
           ABT     3.677909         -0.011216  0.335557  56.962557   
...                     ...               ...       ...        ...   
2023-09-30 OTIS    4.415425         -0.000097 -1.534536  33.116256   
           ABNB    4.940924          0.000213 -0.037854  44.494127   
           CEG     4.692320          0.000131  0.366876  55.245482   
           GEHC    4.212972          0.000185 -1.116463  40.922337   
           KVUE    3.089221         -0.000159 -1.435621  35.706312   

                   dollar_vol_rank  
date       ticker                   
2015-11-30 A                   NaN  
           AAL                 NaN  
           AAPL                NaN  
           ABBV                NaN  
           ABT                 NaN  
...                            ...  
2023-09-30 OTIS                NaN  
           ABNB                NaN  
           CEG                 NaN  
           GEHC                NaN  
           KVUE                NaN  

[46648 rows x 10 columns]